# Data processing with Pandas, part 2

This week we will continue developing our skills using [Pandas](https://pandas.pydata.org/) to process real data. 

## Motivation

![Finland April 2019](img/Finland-April-2019.png)
*Source: [https://weather.com/news/climate/news/2019-05-20-april-2019-global-temperatures-nasa-noaa](https://weather.com/news/climate/news/2019-05-20-april-2019-global-temperatures-nasa-noaa)*

April 2019 was the [second warmest April on record globally](https://weather.com/news/climate/news/2019-05-20-april-2019-global-temperatures-nasa-noaa), and the warmest on record at 13 weather stations in Finland. 
In this lesson, we will use our data manipulation and analysis skills to analyze weather data from Finland, and investigate the claim that April 2019 was the warmest on record across Finland.

Along the way we will cover a number of useful techniques in pandas including:

- renaming columns
- iterating data frame rows and applying functions
- data aggregation
- repeating the analysis task for several input files 

## Input data

In the lesson this week we are using weather observation data from Finland [downloaded from NOAA](https://www7.ncdc.noaa.gov/CDO/cdopoemain.cmd?datasetabbv=DS3505&countryabbv=&georegionabbv=&resolution=40). You will be working with data from either 15 or 4 different weather obsercation stations from Finland, depending on your environment. 

## Downloading the data

The first step for today's lesson is to get the data. Which data files you download will depend on the platform you're using for working through the lesson today.

### CSC Notebooks users

If you're working on the CSC Notebooks platform, you can download the data by opening a new terminal window in Jupyter Lab by going to **File** -> **New** -> **Terminal** in the Jupyter Lab menu bar. Once the terminal is open, you will need to navigate to the directory for Lesson 6 by typing

```bash
cd notebooks/notebooks/L6/
```

Once in the correct directory, you can confirm this by typing

```bash
ls
```

You should see something like the following output:

```bash
advanced-data-processing-with-pandas.ipynb errors.ipynb                               img
debugging.ipynb                            gcp-assertions.ipynb
```

If so, you're in the correct directory and you can download the data files by typing

```bash
wget https://davewhipp.github.io/data/Finland-weather-data-CSC.tar.gz
```

After the download completes, you can extract the data files by typing

```bash
tar zxvf Finland-weather-data-CSC.tar.gz
```

At this stage you should have a new directory called `data` that contains the data for this week's lesson. You can confirm this by typing

```bash
ls data
```

You should see something like the following:

```bash
029440.txt           029720.txt           3505doc.txt          6367598020644stn.txt
029700.txt           029740.txt           6367598020644inv.txt
```

Now you should be all set to proceed with the lesson!

### Users with Jupyter on their personal computers

If you're working on your own computer, you can download the data by opening a new terminal window in Jupyter Lab by going to **File** -> **New** -> **Terminal** in the Jupyter Lab menu bar. Once the terminal is open, you will need to navigate to the directory for Lesson 6 by typing

```bash
cd path/to/L6/
```

where `path/to/` should be replaced with the directories needed to locate the Lesson 6 materials in on your computer. Once in the correct directory, you can confirm this by typing

```bash
ls
```

You should see something like the following output:

```bash
advanced-data-processing-with-pandas.ipynb errors.ipynb                               img
debugging.ipynb                            gcp-assertions.ipynb
```

If so, you're in the correct directory and you can download the data files by typing

```bash
wget https://davewhipp.github.io/data/Finland-weather-data-full.tar.gz
```

After the download completes, you can extract the data files by typing

```bash
tar zxvf Finland-weather-data-full.tar.gz
```

At this stage you should have a new directory called `data` that contains the data for this week's lesson. You can confirm this by typing

```bash
ls data
```

You should see something like the following:

```bash
028360.txt           029070.txt           029440.txt           029740.txt  6367598020644inv.txt
028690.txt           029110.txt           029500.txt           029810.txt  6367598020644stn.txt
028750.txt           029170.txt           029700.txt           029820.txt
028970.txt           029350.txt           029720.txt           3505doc.txt
```

Now you should be all set to proceed with the lesson!

### Binder users

It is not recommended to complete this lesson using Binder.

## About the data

As part of the download there are a number of files that describe the weather data. These *metadata* files include:

- A list of stations\*: [data/6367598020644stn.txt](metadata/6367598020644stn.txt)
- Details about weather observations at each station: [data/6367598020644inv.txt](metadata/6367598020644inv.txt)
- A data description (i.e., column names): [data/3505doc.txt](metadata/3505doc.txt)

\*Note that the list of stations is for all 15 stations, even if you're working with only the 4 stations on the CSC Notebooks platform.

The input data for this week are separated with varying number of spaces (i.e., fixed width). The first lines and columns of the data look like following:

``` 
  USAF  WBAN YR--MODAHRMN DIR SPD GUS CLG SKC L M H  VSB MW MW MW MW AW AW AW AW W TEMP DEWP    SLP   ALT    STP MAX MIN PCP01 PCP06 PCP24 PCPXX SD
029440 99999 190601010600 090   7 *** *** OVC * * *  0.0 ** ** ** ** ** ** ** ** *   27 **** 1011.0 ***** ****** *** *** ***** ***** ***** ***** ** 
029440 99999 190601011300 ***   0 *** *** OVC * * *  0.0 ** ** ** ** ** ** ** ** *   27 **** 1015.5 ***** ****** *** *** ***** ***** ***** ***** ** 
029440 99999 190601012000 ***   0 *** *** OVC * * *  0.0 ** ** ** ** ** ** ** ** *   25 **** 1016.2 ***** ****** *** *** ***** ***** ***** ***** ** 
029440 99999 190601020600 ***   0 *** *** CLR * * *  0.0 ** ** ** ** ** ** ** ** *   26 **** 1016.2 ***** ****** *** *** ***** ***** ***** ***** **
```

We will develop our analysis workflow using data for a single station. Then, we will repeat the same process for all the stations.

## Reading the data

In order to get started, let's import pandas: 

In [1]:
import pandas as pd

At this point, you can already have a quick look at the input file `029440.txt` for Tampere Pirkkala and how it is structured. We can notice at least two things we need to consider when reading in the data:

<div class="alert alert-info">

**NOTE: Input data structure**

- **Delimiter:** The data are **separated with varying amount of spaces**. If you check out the documentation for the [read_csv() method](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html), you can see that there are two different ways of doing this. We can either use the `sep` or `delim_whitespace` parameter;  `sep='\s+'` or `delim_whitespace=True` but not both. In this case, we prefer to use `delim_whitespace`.

- **No Data values:** No data values in the NOAA data are coded with varyingg number of `*`. We can tell pandas to consider those characters as NaNs by specifying `na_values=['*', '**', '***', '****', '*****', '******']`.
</div>

In [2]:
fp = r"data/029440.txt"

# Read data using varying amount of spaces as separator and specifying * characters as NoData values
data = pd.read_csv(fp, delim_whitespace=True, na_values=['*', '**', '***', '****', '*****', '******'])

C:\HYapp\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (29,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


- Let's see how the data looks by printing the first five rows with the `head()` function

In [3]:
print(data.head())

    USAF   WBAN  YR--MODAHRMN    DIR  SPD  GUS  CLG  SKC   L   M ...     SLP  \
0  29440  99999  190601010600   90.0  7.0  NaN  NaN  OVC NaN NaN ...  1011.0   
1  29440  99999  190601011300    NaN  0.0  NaN  NaN  OVC NaN NaN ...  1015.5   
2  29440  99999  190601012000    NaN  0.0  NaN  NaN  OVC NaN NaN ...  1016.2   
3  29440  99999  190601020600    NaN  0.0  NaN  NaN  CLR NaN NaN ...  1016.2   
4  29440  99999  190601021300  270.0  7.0  NaN  NaN  OVC NaN NaN ...  1015.6   

   ALT  STP  MAX  MIN  PCP01  PCP06  PCP24  PCPXX  SD  
0  NaN  NaN  NaN  NaN    NaN    NaN    NaN    NaN NaN  
1  NaN  NaN  NaN  NaN    NaN    NaN    NaN    NaN NaN  
2  NaN  NaN  NaN  NaN    NaN    NaN    NaN    NaN NaN  
3  NaN  NaN  NaN  NaN    NaN    NaN    NaN    NaN NaN  
4  NaN  NaN  NaN  NaN    NaN    NaN    NaN    NaN NaN  

[5 rows x 33 columns]


All seems ok. However, we won't be needing all of the 33 columns for detecting warm temperatures in April. We can check all column names by running `data.columns`:

In [4]:
data.columns

Index(['USAF', 'WBAN', 'YR--MODAHRMN', 'DIR', 'SPD', 'GUS', 'CLG', 'SKC', 'L',
       'M', 'H', 'VSB', 'MW', 'MW.1', 'MW.2', 'MW.3', 'AW', 'AW.1', 'AW.2',
       'AW.3', 'W', 'TEMP', 'DEWP', 'SLP', 'ALT', 'STP', 'MAX', 'MIN', 'PCP01',
       'PCP06', 'PCP24', 'PCPXX', 'SD'],
      dtype='object')

A description for all these columns is available in the metadata file [data/3505doc.txt](metadata/3505doc.txt). 

**Let's read in the data one more time.** This time, we will read in only some of the columns using the `usecols` parameter. Let's read in columns that might be somehow useful to our analysis, or at least that contain some values that are meaningful to us, including the station name, timestamp, and data about wind and temperature: `'USAF','YR--MODAHRMN', 'DIR', 'SPD', 'GUS','TEMP', 'MAX', 'MIN'`

In [5]:
data = pd.read_csv(fp, delim_whitespace=True, usecols=['USAF','YR--MODAHRMN', 'DIR', 'SPD', 'GUS','TEMP', 'MAX', 'MIN'], na_values=['*', '**', '***', '****', '*****', '******'])
data.head()

,USAF,YR--MODAHRMN,DIR,SPD,GUS,TEMP,MAX,MIN
0,29440,190601010600,90.0,7.0,NaN,27.0,NaN,NaN
1,29440,190601011300,NaN,0.0,NaN,27.0,NaN,NaN
2,29440,190601012000,NaN,0.0,NaN,25.0,NaN,NaN
3,29440,190601020600,NaN,0.0,NaN,26.0,NaN,NaN
4,29440,190601021300,270.0,7.0,NaN,27.0,NaN,NaN


Okay so we can see that the data was successfully read to the DataFrame and we also seemed to be able to convert the asterisk (\*) characters into `NaN` values. 

## Renaming columns

Check again the column names in our DataFrame:

In [6]:
data.columns

Index(['USAF', 'YR--MODAHRMN', 'DIR', 'SPD', 'GUS', 'TEMP', 'MAX', 'MIN'], dtype='object')

The column names that we have are somewhat ackward. Let's change them into more intuitive ones. 
This can be done easily using the `rename()` method and a dictionary that lists old and new column names.

<div class="alert alert-info">

**Dictionaries**

[Dictionary](https://docs.python.org/2/tutorial/datastructures.html#dictionaries) is a spesific data structure in Python for storing key-value pairs. During this course, we will use dictionaries mainly when renaming columns in a pandas series, but dictionaries are useful for many different purposes! For more information about Python dictionaries, check out [this tutorial](https://realpython.com/python-dicts/).
    
</div>



We can define the new column names using a [dictionary](https://www.tutorialspoint.com/python/python_dictionary.htm) where we determine "`key: value`" -pairs, in which the original column name (the one which will be replaced) is the key, and the new column name is the value.

- Let's change:
   
   - `YR--MODAHRMN` column into `TIME`, 
   - `SPD` into `SPEED`, and
   - `GUS` into `GUST`

In [7]:
# Create the dictionary with old and new names
new_names = {'YR--MODAHRMN': 'TIME', 'SPD': 'SPEED', 'GUS': 'GUST'}

# Let's see what they look like and what is the type
print(new_names)
print(type(new_names))

{'SPD': 'SPEED', 'YR--MODAHRMN': 'TIME', 'GUS': 'GUST'}
<class 'dict'>


From above we can see that we have successfully created a dictionary that is of type `dict`. 

- Now we can change the column names by passing that dictionary into parameter `columns` in `rename()` -function.

In [8]:
# Rename the columns
data = data.rename(columns=new_names)

# Print the new columns
print(data.columns)

Index(['USAF', 'TIME', 'DIR', 'SPEED', 'GUST', 'TEMP', 'MAX', 'MIN'], dtype='object')


Perfect, now our column names are more easy to understand and use. 

<div class="alert alert-info">

**TASK: Renaming columns**

The temperature values are again in Fahrenheit. As you might guess, we will soon convert these temperatures in to Celsius. In order to avoid confusion with the columns, rename column `TEMP` into `TEMP_F`. Also, we could rename `USAF` as`STATION_NUMBER`.
</div>


In [9]:
# Create the dictionary with old and new names
new_names = {'USAF':'STATION_NUMBER', 'TEMP': 'TEMP_F'}

# Rename the columns
data = data.rename(columns=new_names)

# Check the output
data.head()

,STATION_NUMBER,TIME,DIR,SPEED,GUST,TEMP_F,MAX,MIN
0,29440,190601010600,90.0,7.0,NaN,27.0,NaN,NaN
1,29440,190601011300,NaN,0.0,NaN,27.0,NaN,NaN
2,29440,190601012000,NaN,0.0,NaN,25.0,NaN,NaN
3,29440,190601020600,NaN,0.0,NaN,26.0,NaN,NaN
4,29440,190601021300,270.0,7.0,NaN,27.0,NaN,NaN


## Check dataframe properties

As we learned last week, it's always a good idea to check basic properties of the input data before proceeding with data analysis. Let's check:

- How many rows and columns we have:

In [10]:
data.shape

(757983, 8)

- Top and bottom rows: 

In [11]:
data.head()

,STATION_NUMBER,TIME,DIR,SPEED,GUST,TEMP_F,MAX,MIN
0,29440,190601010600,90.0,7.0,NaN,27.0,NaN,NaN
1,29440,190601011300,NaN,0.0,NaN,27.0,NaN,NaN
2,29440,190601012000,NaN,0.0,NaN,25.0,NaN,NaN
3,29440,190601020600,NaN,0.0,NaN,26.0,NaN,NaN
4,29440,190601021300,270.0,7.0,NaN,27.0,NaN,NaN


In [12]:
data.tail()

,STATION_NUMBER,TIME,DIR,SPEED,GUST,TEMP_F,MAX,MIN
757978,29440,201910012220,130.0,3.0,NaN,39.0,NaN,NaN
757979,29440,201910012250,110.0,3.0,NaN,37.0,NaN,NaN
757980,29440,201910012300,100.0,2.0,NaN,38.0,NaN,NaN
757981,29440,201910012320,100.0,3.0,NaN,37.0,NaN,NaN
757982,29440,201910012350,110.0,3.0,NaN,37.0,NaN,NaN


- Data types of the columns: 

In [13]:
data.dtypes

STATION_NUMBER      int64
TIME                int64
DIR               float64
SPEED             float64
GUST              float64
TEMP_F            float64
MAX               float64
MIN               float64
dtype: object

- Descriptive statistics:

In [14]:
print(data.describe())

       STATION_NUMBER          TIME            DIR          SPEED  \
count        757983.0  7.579830e+05  699256.000000  750143.000000   
mean          29440.0  1.999974e+11     233.499846       6.742641   
std               0.0  1.629544e+09     209.707258       4.296191   
min           29440.0  1.906010e+11      10.000000       0.000000   
25%           29440.0  1.989083e+11     130.000000       3.000000   
50%           29440.0  2.004042e+11     200.000000       7.000000   
75%           29440.0  2.012050e+11     270.000000       9.000000   
max           29440.0  2.019100e+11     990.000000      61.000000   

               GUST         TEMP_F           MAX           MIN  
count  19906.000000  754862.000000  23869.000000  23268.000000  
mean      20.147996      40.409778     45.373539     35.783737  
std        7.415138      17.898715     18.242679     17.195427  
min       11.000000     -33.000000    -26.000000    -32.000000  
25%       14.000000      29.000000     32.000000     

Here we can see that there are varying number of observations per column (see the `count` information), because some of the columns have missing values.

## Using your own functions in pandas 

Now it's again time to convert temperatures from Fahrenheit to Celsius! Yes, we have already done this many times before, but this time we will learn how to apply self-made functions to data in a pandas DataFrame.
**In short, our task is to define a function for the temperature conversion, and to apply this function for each Celsius value on each row of the DataFrame. Output celsius values should be stored in a new column called** `TEMP_C`.

Knowing how to use your own function in pandas can be really useful when doing your own analyses. Here, we will introduce two different approaches for using function in pandas. First, we will see how we can apply the function row-by-row using a `for`-loop and the `DataFrame.iterrows()`-method, and then we will learn how to apply the method to all rows at once using [DataFrame.apply](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.apply.html).

For both of these approaches, we first need to define our temperature conversion function from Fahrenheit to Celsius:

In [15]:
def fahr_to_celsius(temp_fahrenheit):
    """
    Function to convert Fahrenheit temperature into Celsius.

    Parameters
    ----------

    temp_fahrenheit: int | float
        Input temperature in Fahrenheit (should be a number)
        
    Returns
    -------
    
    Temperature in Celsius (float)
    """

    # Convert the Fahrenheit into Celsius and return it
    converted_temp = (temp_fahrenheit - 32) / 1.8
    return converted_temp

**Note:** with such a simple example, we could use the function direcly on a column in the DataFrame in order to conver the values:

In [16]:
data["TEMP_C"] = fahr_to_celsius(data["TEMP_F"])
data.head()

,STATION_NUMBER,TIME,DIR,SPEED,GUST,TEMP_F,MAX,MIN,TEMP_C
0,29440,190601010600,90.0,7.0,NaN,27.0,NaN,NaN,-2.777778
1,29440,190601011300,NaN,0.0,NaN,27.0,NaN,NaN,-2.777778
2,29440,190601012000,NaN,0.0,NaN,25.0,NaN,NaN,-3.888889
3,29440,190601020600,NaN,0.0,NaN,26.0,NaN,NaN,-3.333333
4,29440,190601021300,270.0,7.0,NaN,27.0,NaN,NaN,-2.777778


If we want to do something more complicated, we need to know how to apply the function row-by-row.

### Iterating over rows

We can iterate over the rows of Pandas DataFrame by using the `iterrows()` -method and use the function one row at a time.


When iterating over the rows in our `DataFrame`, it is noteworthy to understand that pandas actually keeps track on the `index` value as well. Hence, the contents of a single row actually contains not only the values, but also the `index` of that row (each row is a pandas Series!). 

- Let's see how `iterrows()` works by printing out the `TEMP` value on each row using a `for`-loop:


In [17]:
# Iterate over the rows
for idx, row in data.iterrows():
    # Print the index value
    print('Index:', idx)
    
    # Print the row
    print('Temp F:', row["TEMP_F"], "\n")
    
    break

Index: 0
Temp F: 27.0 



<div class="alert alert-info">

**break**

When developing a for-loop, you don't always need to go trough the whole loop if you just want to test things out. 
[break](https://www.tutorialspoint.com/python/python_break_statement.htm) statement in Python terminates the current loop after the first iteration and we used it here just to test check out the values on the first row.
With a large data, you might not want to print out thousands of values to the screen!
    
</div>


We can see that the `idx` variable indeed contains the index value at position 0 (the first row) and the `row` variable contains all the data from that given row stored as a pandas `Series`.

- Let's now create an empty column `TEMP_C` for the Celsius temperatures and update the values into that column using the `fahr_to_celsius` function we defined earlier:


In [18]:
# Create an empty column for the DataFrame where the values will be stored
new_column = "TEMP_C"
data[new_column] = None

# Iterate over the rows 
for idx, row in data.iterrows():
    # Convert the Fahrenheit to Celsius
    celsius = fahr_to_celsius(row['TEMP_F'])
    
    # Update the value of 'Celsius' column with the converted value
    data.at[idx, new_column] = celsius

<div class="alert alert-info">

**Reminder: .at or .loc?**

Here, you could also use `data.loc[idx, new_column] = celsius` to achieve the same result. 
    
If you only need to access a single value in a DataFrame, [DataFrame.at](https://pandas.pydata.org/pandas-docs/version/0.25/reference/api/pandas.DataFrame.at.html) is faster compared to [DataFrame.loc](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.loc.html) which is designed for accessing groups of rows and columns. 

Check out more examples for using `.at` and `.loc` from lesson 5 materials.
    
</div>

- Let's see what we have now.

In [19]:
data.head(10)

,STATION_NUMBER,TIME,DIR,SPEED,GUST,TEMP_F,MAX,MIN,TEMP_C
0,29440,190601010600,90.0,7.0,NaN,27.0,NaN,NaN,-2.77778
1,29440,190601011300,NaN,0.0,NaN,27.0,NaN,NaN,-2.77778
2,29440,190601012000,NaN,0.0,NaN,25.0,NaN,NaN,-3.88889
3,29440,190601020600,NaN,0.0,NaN,26.0,NaN,NaN,-3.33333
4,29440,190601021300,270.0,7.0,NaN,27.0,NaN,NaN,-2.77778
5,29440,190601022000,NaN,0.0,NaN,27.0,NaN,NaN,-2.77778
6,29440,190601030600,270.0,7.0,NaN,26.0,NaN,NaN,-3.33333
7,29440,190601031300,270.0,7.0,NaN,25.0,NaN,NaN,-3.88889
8,29440,190601032000,270.0,7.0,NaN,24.0,NaN,NaN,-4.44444
9,29440,190601040600,NaN,0.0,NaN,18.0,NaN,NaN,-7.77778


Great! Now we have converted our temperatures into Celsius by using our self-made function.

### Applying a function

Pandas DataFrames and Series also have a dedicated method `.apply()` for applying functions on columns (or rows!). When using `.apply()`, we pass the function name (without parenthesis!) as an argument:

In [20]:
data["TEMP_C"] = data["TEMP_F"].apply(fahr_to_celsius)
data.head()

,STATION_NUMBER,TIME,DIR,SPEED,GUST,TEMP_F,MAX,MIN,TEMP_C
0,29440,190601010600,90.0,7.0,NaN,27.0,NaN,NaN,-2.777778
1,29440,190601011300,NaN,0.0,NaN,27.0,NaN,NaN,-2.777778
2,29440,190601012000,NaN,0.0,NaN,25.0,NaN,NaN,-3.888889
3,29440,190601020600,NaN,0.0,NaN,26.0,NaN,NaN,-3.333333
4,29440,190601021300,270.0,7.0,NaN,27.0,NaN,NaN,-2.777778


**Note:** pay attention which column you are applying the function on! Running this code: `data.apply(fahr_to_celsius)` would not give an error, but the results also don't make much sense.

<div class="alert alert-info">

**Should I use .iterrows() or .apply()?**

We are teaching the `.iterrows()` method because it helps to understand the structure of a DataFrame and the process of looping trough DataFrame rows. However, using `.apply()` is often more efficient in terms of execution time. 
    
    
At this point, the most important thing is that you understand what happens when you are modifying the values in a pandas DataFrame. When doing the course exercises, either of these approaches is ok!
    
</div>

## Parsing dates

We will eventually want to group our data based on month in order to see if April temperatures in 2019 were higher than average. Currently, the date and time information is stored in the column `TIME`:

`YR--MODAHRMN = YEAR-MONTH-DAY-HOUR-MINUTE IN GREENWICH MEAN TIME (GMT)`

Let's have a closer look at the date and time information we have by checking the values in that column, and their data type:

In [21]:
data["TIME"].head(10)

0    190601010600
1    190601011300
2    190601012000
3    190601020600
4    190601021300
5    190601022000
6    190601030600
7    190601031300
8    190601032000
9    190601040600
Name: TIME, dtype: int64

In [22]:
data["TIME"].tail(10)

757973    201910012050
757974    201910012100
757975    201910012120
757976    201910012150
757977    201910012200
757978    201910012220
757979    201910012250
757980    201910012300
757981    201910012320
757982    201910012350
Name: TIME, dtype: int64

The `TIME` column contains several observations per day (and even several observations per hour). The timestamp for the first observation is `190601010600`, i.e. from 1st of January 1906 (way back!), and the timestamp for the latest observation is `201910012350` (from last week, by the time of writing this!)

In [23]:
data["TIME"].dtypes

dtype('int64')

And the information is stored as integer values.

There are several different options for proceeding from here. The bottom line is, that we would want to **aggregate the data on a monthly level**, and in order to do so we need to "label" each row of data based on the month when the record was observed. 
In practice, we could create a new column (or an index), which contains information about the month (including the year, but excluding days, hours and minutes).

### String slicing

One approach would be to convert the date and time information into character strings and "cut" the needed information from the [string objects](https://docs.python.org/3/tutorial/introduction.html#strings). If we look at the latest time stamp in the data (`201910012350`), you can see that there is a systematic pattern `YEAR-MONTH-DAY-HOUR-MINUTE`. Four first characters represent the year, and six first characters are year + month!

In [24]:
date = "201910012350"
date[0:6]

'201910'

Doing this in pandas requires two steps:
  1. Convert the `TIME` column from `int` into `str` datatype.
  2. Slice the correct range of characters from the character string using [pandas.Series.str.slice()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.slice.html)

- Let's convert the time into string. And check that the data type changes:

In [25]:
# Convert to string
data['TIME_STR'] = data['TIME'].astype(str)

In [26]:
# SLice the string
data['YEAR_MONTH'] = data['TIME_STR'].str.slice(start=0, stop=6)

# Let's see what we have
print(data.head())

   STATION_NUMBER          TIME    DIR  SPEED  GUST  TEMP_F  MAX  MIN  \
0           29440  190601010600   90.0    7.0   NaN    27.0  NaN  NaN   
1           29440  190601011300    NaN    0.0   NaN    27.0  NaN  NaN   
2           29440  190601012000    NaN    0.0   NaN    25.0  NaN  NaN   
3           29440  190601020600    NaN    0.0   NaN    26.0  NaN  NaN   
4           29440  190601021300  270.0    7.0   NaN    27.0  NaN  NaN   

     TEMP_C      TIME_STR YEAR_MONTH  
0 -2.777778  190601010600     190601  
1 -2.777778  190601011300     190601  
2 -3.888889  190601012000     190601  
3 -3.333333  190601020600     190601  
4 -2.777778  190601021300     190601  


Nice! Now we have "labeled" the rows based on information about day of the year and hour of the day. However, let's have a look at a more clever way of dealing with dates and times..

### datetime

<div class="alert alert-info">

**Pandas datetime**

In pandas, we can convert dates and times into a new data type [datetime](https://docs.python.org/3.7/library/datetime.html) using [pandas.to_datetime](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html) function. First, it is important to understand the structure of the input data in order to avoid erroneous conversions, and that's why we first learned string slicing before introducing the datetime functionalities. 
    
Here is one example of how to convert the `TIME_STR`-column in our data set to datetime:
    
```
# Convert to datetime
data["DATE"] = pd.to_datetime(data["TIME_STR"])

```
   
        
If needed, you can use the `format` parameter to define the output datetime format according to [strftime(format) method](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior). together with `exact=False`, for example like this: 
    
```
# Convert to datetime
data["YEAR_MONTH"] = pd.to_datetime(data["TIME_STR"], format='%Y%m', exact=False)

```
In this example, `exact=False` drops out days, hours and minutes, because they are not included in the specified formatting.
 
</div>




In [27]:
# Convert to datetime
data["DATE"] = pd.to_datetime(data["TIME_STR"])
data["DATE"].head()

0   1906-01-01 06:00:00
1   1906-01-01 13:00:00
2   1906-01-01 20:00:00
3   1906-01-02 06:00:00
4   1906-01-02 13:00:00
Name: DATE, dtype: datetime64[ns]

Note: in this case, the data type of the values is `datetime`. 

<div class="alert alert-info">

**Pandas Series datetime properties**
    
There are several methods available for accessing information about the properties of datetime values. Read more from the pandas documentation about [datetime properties](https://pandas.pydata.org/pandas-docs/stable/reference/series.html#datetime-properties).
</div>

Now, we can extract different time units based on the datetime-column using the [pandas.Series.dt accessor](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html):

In [28]:
data['DATE'].dt.year

0         1906
1         1906
2         1906
3         1906
4         1906
5         1906
6         1906
7         1906
8         1906
9         1906
10        1906
11        1906
12        1906
13        1906
14        1906
15        1906
16        1906
17        1906
18        1906
19        1906
20        1906
21        1906
22        1906
23        1906
24        1906
25        1906
26        1906
27        1906
28        1906
29        1906
          ... 
757953    2019
757954    2019
757955    2019
757956    2019
757957    2019
757958    2019
757959    2019
757960    2019
757961    2019
757962    2019
757963    2019
757964    2019
757965    2019
757966    2019
757967    2019
757968    2019
757969    2019
757970    2019
757971    2019
757972    2019
757973    2019
757974    2019
757975    2019
757976    2019
757977    2019
757978    2019
757979    2019
757980    2019
757981    2019
757982    2019
Name: DATE, Length: 757983, dtype: int64

We can also combine the datetime functionalities with other methods from pandas. For example, we can check the number of unique years in our input data: 

In [29]:
data['DATE'].dt.year.nunique()

51

<div class="alert alert-info">

**TASK:**

- Create two new columns: `YEAR` and `MONTH` based on the date column
</div>

In [30]:
data['YEAR'] = data['DATE'].dt.year
data['MONTH'] = data['DATE'].dt.month

data.head()

,STATION_NUMBER,TIME,DIR,SPEED,GUST,TEMP_F,MAX,MIN,TEMP_C,TIME_STR,YEAR_MONTH,DATE,YEAR,MONTH
0,29440,190601010600,90.0,7.0,NaN,27.0,NaN,NaN,-2.777778,190601010600,190601,1906-01-01 06:00:00,1906,1
1,29440,190601011300,NaN,0.0,NaN,27.0,NaN,NaN,-2.777778,190601011300,190601,1906-01-01 13:00:00,1906,1
2,29440,190601012000,NaN,0.0,NaN,25.0,NaN,NaN,-3.888889,190601012000,190601,1906-01-01 20:00:00,1906,1
3,29440,190601020600,NaN,0.0,NaN,26.0,NaN,NaN,-3.333333,190601020600,190601,1906-01-02 06:00:00,1906,1
4,29440,190601021300,270.0,7.0,NaN,27.0,NaN,NaN,-2.777778,190601021300,190601,1906-01-02 13:00:00,1906,1


## Aggregating data in Pandas by grouping

Here, we will learn how to use [pandas.DataFrame.groupby](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html) which is a handy method for compressing large amounts of data and computing statistics for subgroups.

Our practical task is to calculate the average temperatures for each month

This can be done by aggregating the data, i.e.:

  1. **grouping the data** based on year and month
  2. Calculating the average for each month (each group) either by using a for-loop or directly from the grouped object
  3. Storing those values into **a new DataFrame** `monthly_data`

Before we start grouping the data, let's once more check how our input data looks like:

In [31]:
print("number of rows:", len(data))
data.head()

number of rows: 757983


,STATION_NUMBER,TIME,DIR,SPEED,GUST,TEMP_F,MAX,MIN,TEMP_C,TIME_STR,YEAR_MONTH,DATE,YEAR,MONTH
0,29440,190601010600,90.0,7.0,NaN,27.0,NaN,NaN,-2.777778,190601010600,190601,1906-01-01 06:00:00,1906,1
1,29440,190601011300,NaN,0.0,NaN,27.0,NaN,NaN,-2.777778,190601011300,190601,1906-01-01 13:00:00,1906,1
2,29440,190601012000,NaN,0.0,NaN,25.0,NaN,NaN,-3.888889,190601012000,190601,1906-01-01 20:00:00,1906,1
3,29440,190601020600,NaN,0.0,NaN,26.0,NaN,NaN,-3.333333,190601020600,190601,1906-01-02 06:00:00,1906,1
4,29440,190601021300,270.0,7.0,NaN,27.0,NaN,NaN,-2.777778,190601021300,190601,1906-01-02 13:00:00,1906,1


We have quite a few rows of weather data, and several observations per day. **Our goal is to create an aggreated data frame that would have only one row per month!**

Let's **group** our data based on unique year and month combinations

In [32]:
grouped = data.groupby(["YEAR", "MONTH"])

<div class="alert alert-info">

**NOTE:**
    
Here you could also group the data based on the `YEAR_MONTH` column to achieve the same result:
    
```
# Group the data 
grouped = data.groupby('YEAR_MONTH')
```
</div>

- Let's see what we have now.

In [33]:
# What is the type?
print("Type:\n", type(grouped))

# How many?
print("Length:\n", len(grouped))

Type:
 <class 'pandas.core.groupby.groupby.DataFrameGroupBy'>
Length:
 601


Okey, interesting. Now we have a new object with type `DataFrameGroupBy`.

<div class="alert alert-info">

**TASK:**

Think: what does the number of groups (length of the grouped object) tell us?
</div>

In [34]:
# Answer: the length of the grouped object should be the same as
data["YEAR_MONTH"].nunique()

# in other words, the number of groups is the number of unique year and month combinations in our data

601

There are several methods we can use for extracting information from the grouped data. See [documentation for Pandas GroupBy objects](https://pandas.pydata.org/pandas-docs/stable/reference/groupby.html) for a comprehensive overview. 

**Checking group names:**

In [35]:
# Check the "names" of each group (uncomment the next row if you want to print out all the keys)
#grouped.groups.keys()

**Accessing data for one group:**

- Let's check the contents for a group representing August 2019 (name of that group is `(2019, 4)` if you grouped the data based on datetime columns `YEAR` and `MONTH`). We can get the values of that hour from the grouped object using the `get_group()` -method:

In [36]:
# Specify the time of the first hour (as text)
month = (2019, 4)

# Select the group
group1 = grouped.get_group(month)

# Let's see what we have
print(group1)

        STATION_NUMBER          TIME    DIR  SPEED  GUST  TEMP_F   MAX   MIN  \
745098           29440  201904010000  280.0    6.0   NaN    30.0   NaN   NaN   
745099           29440  201904010020  280.0    7.0   NaN    32.0   NaN   NaN   
745100           29440  201904010050  280.0    6.0   NaN    30.0   NaN   NaN   
745101           29440  201904010100  280.0    7.0   NaN    30.0   NaN   NaN   
745102           29440  201904010120  280.0    6.0   NaN    30.0   NaN   NaN   
745103           29440  201904010150  300.0    5.0   NaN    30.0   NaN   NaN   
745104           29440  201904010200  290.0    6.0   NaN    30.0   NaN   NaN   
745105           29440  201904010220  290.0    6.0   NaN    30.0   NaN   NaN   
745106           29440  201904010250  290.0    6.0   NaN    28.0   NaN   NaN   
745107           29440  201904010300  290.0    5.0   NaN    29.0   NaN   NaN   
745108           29440  201904010320  280.0    6.0   NaN    30.0   NaN   NaN   
745109           29440  201904010350  28

Ahaa! As we can see, a single group contains a **DataFrame** with values only for that specific month. Let's check the DataType of this group:

In [37]:
type(group1)

pandas.core.frame.DataFrame

So, one group is a pandas DataFrame! This is really useful, because we can now use all the familiar DataFrame methods for calculating statistics etc for this spesific group. 
We can, for example, calculate the average values for all variables using the statistical functions that we have seen already (e.g. mean, std, min, max, median, etc.).

We can do that by using the `mean()` -function that we already used during the Lesson 5. 

- Let's calculate the mean for following attributes all at once:
   - `DIR`, 
   - `SPEED`, 
   - `GUST`, 
   - `TEMP`, 
   - `TEMP_C`
   - `MONTH` 

In [38]:
# Specify the columns that will be part of the calculation
mean_cols = ['DIR', 'SPEED', 'GUST', 'TEMP_F', 'TEMP_C', 'MONTH']

# Calculate the mean values all at one go
mean_values = group1[mean_cols].mean()

# Let's see what we have
print(mean_values)

DIR       309.035306
SPEED       5.932188
GUST       15.868217
TEMP_F     42.472030
TEMP_C      5.817794
MONTH       4.000000
dtype: float64


Here we saw how you can access data from a single group. For getting information about all groups (all months) we can a `for` -loop or methods available in the grouped object.

**For-loops and grouped objects:**

When iterating over the groups in our `DataFrameGroupBy` -object
it is important to understand that a single group in our `DataFrameGroupBy` actually contains not only the actual values, but also information about the `key` that was used to do the grouping. Hence, when iterating over the data we need to assign the `key` and the values into separate variables.

- Let's see how we can iterate over the groups and print the key and the data from a single group (again using `break` to only see what is happening).


In [39]:
# Iterate over groups
for key, group in grouped:
    # Print key and group
    print("Key:\n", key)
    print("\nFirst rows of data in this group:\n", group.head())
    
    # Stop iteration with break command
    break

Key:
 (1906, 1)

First rows of data in this group:
    STATION_NUMBER          TIME    DIR  SPEED  GUST  TEMP_F  MAX  MIN  \
0           29440  190601010600   90.0    7.0   NaN    27.0  NaN  NaN   
1           29440  190601011300    NaN    0.0   NaN    27.0  NaN  NaN   
2           29440  190601012000    NaN    0.0   NaN    25.0  NaN  NaN   
3           29440  190601020600    NaN    0.0   NaN    26.0  NaN  NaN   
4           29440  190601021300  270.0    7.0   NaN    27.0  NaN  NaN   

     TEMP_C      TIME_STR YEAR_MONTH                DATE  YEAR  MONTH  
0 -2.777778  190601010600     190601 1906-01-01 06:00:00  1906      1  
1 -2.777778  190601011300     190601 1906-01-01 13:00:00  1906      1  
2 -3.888889  190601012000     190601 1906-01-01 20:00:00  1906      1  
3 -3.333333  190601020600     190601 1906-01-02 06:00:00  1906      1  
4 -2.777778  190601021300     190601 1906-01-02 13:00:00  1906      1  


Okey so from here we can see that the `key` contains the name of the group (year, month).

- Let's see how we can create a DataFrame where we calculate the mean values for all those weather attributes that we were interested in. I will repeat slightly the earlier steps so that you can see and better understand what is happening.

In [40]:
# Create an empty DataFrame for the aggregated values
monthly_data = pd.DataFrame()

# The columns that we want to aggregate
mean_cols = ['DIR', 'SPEED', 'GUST', 'TEMP_F', 'TEMP_C', "MONTH"]

# Iterate over the groups
for key, group in grouped:
    
   # Calculate mean
   mean_values = group[mean_cols].mean()

   # Add the ´key´ (i.e. the date+time information) into the aggregated values
   mean_values['YEAR_MONTH'] = key

   # Append the aggregated values into the DataFrame
   monthly_data = monthly_data.append(mean_values, ignore_index=True)

- Let's see what we have now:

In [41]:
print(monthly_data)

            DIR       GUST  MONTH      SPEED     TEMP_C     TEMP_F  YEAR_MONTH
0    218.181818        NaN    1.0  13.204301  -3.596177  25.526882   (1906, 1)
1    178.095238        NaN    2.0  13.142857  -3.445767  25.797619   (1906, 2)
2    232.043011        NaN    3.0  15.021505  -5.107527  22.806452   (1906, 3)
3    232.045455        NaN    4.0  13.811111   3.790123  38.822222   (1906, 4)
4    192.820513        NaN    5.0  10.333333  13.070490  55.526882   (1906, 5)
5    234.222222        NaN    6.0  12.922222  16.246914  61.244444   (1906, 6)
6    226.923077        NaN    7.0  10.827957  18.805257  65.849462   (1906, 7)
7    251.627907        NaN    8.0  11.623656  13.590203  56.462366   (1906, 8)
8    236.986301        NaN    9.0   9.988889   8.611111  47.500000   (1906, 9)
9    199.397590        NaN   10.0  12.365591   4.360812  39.849462  (1906, 10)
10   199.166667        NaN   11.0  14.211111   0.296296  32.533333  (1906, 11)
11   198.636364        NaN   12.0  15.516129  -4.086

Awesome! Now we have aggregated our data and we have a new DataFrame called `monthly_data` where we have mean values for each month in the data set.

**Mean for all groups at once**

We can also achieve the same result by computing the mean of all columns for all groups in the grouped object:

In [42]:
grouped.mean()

STATION_NUMBER          TIME         DIR      SPEED       GUST  \
YEAR MONTH                                                                   
1906 1             29440.0  1.906012e+11  218.181818  13.204301        NaN   
     2             29440.0  1.906021e+11  178.095238  13.142857        NaN   
     3             29440.0  1.906032e+11  232.043011  15.021505        NaN   
     4             29440.0  1.906042e+11  232.045455  13.811111        NaN   
     5             29440.0  1.906052e+11  192.820513  10.333333        NaN   
     6             29440.0  1.906062e+11  234.222222  12.922222        NaN   
     7             29440.0  1.906072e+11  226.923077  10.827957        NaN   
     8             29440.0  1.906082e+11  251.627907  11.623656        NaN   
     9             29440.0  1.906092e+11  236.986301   9.988889        NaN   
     10            29440.0  1.906102e+11  199.397590  12.365591        NaN   
     11            29440.0  1.906112e+11  199.166667  14.211111        NaN   
     12            29440.0  1.906122e+11  198.636364  15.516129        NaN   
1907 1             29440.0  1.907012e+11  190.930233  16.344086        NaN   
     2             29440.0  1.907021e+11  223.466667  15.108434        NaN   
     3             29440.0  1.907032e+11  238.051948  10.763441        NaN   
     4             29440.0  1.907042e+11  201.038961  10.433333        NaN   
     5             29440.0  1.907052e+11  244.823529  12.913978        NaN   
     6             29440.0  1.907062e+11  202.674419  11.633333        NaN   
     7             29440.0  1.907072e+11  215.238095  10.473118        NaN   
     8             29440.0  1.907082e+11  221.139241   9.645161        NaN   
     9             29440.0  1.907092e+11  258.953488  16.466667        NaN   
     10            29440.0  1.907102e+11  192.682927  10.408602        NaN   
     11            29440.0  1.907112e+11  204.222222  15.744444        NaN   
     12            29440.0  1.907122e+11  166.052632  11.000000        NaN   
1908 1             29440.0  1.908012e+11  221.547619  11.376344        NaN   
     2             29440.0  1.908022e+11  190.864198  10.546512        NaN   
     3             29440.0  1.908032e+11  169.315068   6.537634        NaN   
     4             29440.0  1.908042e+11  195.333333   6.222222        NaN   
     5             29440.0  1.908052e+11  259.718310   6.709677        NaN   
     6             29440.0  1.908062e+11  249.651163  20.011111        NaN   
...                    ...           ...         ...        ...        ...   
2017 5             29440.0  2.017052e+11  340.875980   6.872531  15.628492   
     6             29440.0  2.017062e+11  379.561190   6.904539  17.014451   
     7             29440.0  2.017072e+11  359.269933   6.318477  15.245136   
     8             29440.0  2.017082e+11  331.270588   6.489526  15.627178   
     9             29440.0  2.017092e+11  304.676641   5.399907  14.788660   
     10            29440.0  2.017102e+11  277.448502   6.535569  16.598854   
     11            29440.0  2.017112e+11  215.835272   7.165504  16.026846   
     12            29440.0  2.017122e+11  237.338195   7.908435  15.927536   
2018 1             29440.0  2.018012e+11  269.553613   6.898274  17.351792   
     2             29440.0  2.018021e+11  224.710831   5.870871  14.937500   
     3             29440.0  2.018032e+11  292.265435   6.066968  15.392857   
     4             29440.0  2.018042e+11  350.141509   5.704087  15.216981   
     5             29440.0  2.018052e+11  422.810552   5.387764  14.819095   
     6             29440.0  2.018062e+11  445.827233   7.056325  18.480337   
     7             29440.0  2.018072e+11  408.614487   5.607442  15.902778   
     8             29440.0  2.018082e+11  396.655422   6.515248  16.764179   
     9             29440.0  2.018092e+11  290.531856   7.879109  20.581481   
     10            29440.0  2.018102e+11  256.174656   7.452370  17.902685   
     11            29440.0  2.0181

## Detecting warm months

Now, we have aggregated our data on monthly level and all we need to do is to check which years had the warmest April temperatures. A simple approach is to select all aprils from the data, group the data and check which group(s) have the highest mean value:

- select all records that are from April (regardless of the year):

In [43]:
aprils = data[data["MONTH"]==4]

- take a subset of columns that might contain interesting information:

In [44]:
aprils = aprils[['STATION_NUMBER','TEMP_F', 'TEMP_C','YEAR', 'MONTH']]

- group by year and month:

In [45]:
grouped = aprils.groupby(by=["YEAR", "MONTH"])

- calculate mean for each group:

In [46]:
monthly_mean = grouped.mean()
monthly_mean.head()

,,STATION_NUMBER,TEMP_F,TEMP_C
YEAR,MONTH,,,
1906,4,29440,38.822222,3.790123
1907,4,29440,36.111111,2.283951
1908,4,29440,36.811111,2.672840
1909,4,29440,31.977778,-0.012346
1910,4,29440,39.833333,4.351852


- check the highest temperature values (sort the data frame in a descending order):

In [47]:
monthly_mean.sort_values(by="TEMP_C", ascending=False).head(10)

,,STATION_NUMBER,TEMP_F,TEMP_C
YEAR,MONTH,,,
2019,4,29440,42.472030,5.817794
1990,4,29440,41.918084,5.510047
1989,4,29440,41.369647,5.205360
2011,4,29440,41.290730,5.161517
2004,4,29440,41.249676,5.138709
2002,4,29440,41.132353,5.073529
1983,4,29440,41.016183,5.008991
2008,4,29440,40.962343,4.979079
2000,4,29440,40.777778,4.876543


How did April 2019 rank at the Tampere Pirkkala observation station 🌡️? 

## Repeating the data analysis with larger dataset


Finally, let's repeat the data analysis steps above for all the available data we have (!!). First, confirm the path to the **folder** where all the input data are located. 
The idea is, that we will repeat the analysis process for each input file using a (rather long) for loop! Here we have all the main analysis steps with some additional output info - all in one long code cell:

In [48]:
# Read selected columns of  data using varying amount of spaces as separator and specifying * characters as NoData values
data = pd.read_csv(fp, delim_whitespace=True, usecols=['USAF','YR--MODAHRMN', 'DIR', 'SPD', 'GUS','TEMP', 'MAX', 'MIN'], na_values=['*', '**', '***', '****', '*****', '******'])

# Rename the columns
new_names = {'USAF':'STATION_NUMBER','YR--MODAHRMN': 'TIME', 'SPD': 'SPEED', 'GUS': 'GUST', 'TEMP':'TEMP_F'}
data = data.rename(columns=new_names)

#Print info about the current input file:
print("STATION NUMBER:", data.at[0,"STATION_NUMBER"])
print("NUMBER OF OBSERVATIONS:", len(data))

# Create column
col_name = 'TEMP_C'
data[col_name] = None

# Convert tempetarues from Fahrenheits to Celsius
data["TEMP_C"] = data["TEMP_F"].apply(fahr_to_celsius)

# Convert TIME to string 
data['TIME_STR'] = data['TIME'].astype(str)

# Parse year and month
data['MONTH'] = data['TIME_STR'].str.slice(start=5, stop=6).astype(int)
data['YEAR'] = data['TIME_STR'].str.slice(start=0, stop=4).astype(int)

# Extract observations for the months of April 
aprils = data[data['MONTH']==4]

# Take a subset of columns
aprils = aprils[['STATION_NUMBER','TEMP_F', 'TEMP_C', 'YEAR', 'MONTH']]

# Group by year and month
grouped = aprils.groupby(by=["YEAR", "MONTH"])

# Get mean values for each group
monthly_mean = grouped.mean()

# Print info
print(monthly_mean.sort_values(by="TEMP_C", ascending=False).head(5))
print("\n")

STATION NUMBER: 29440
NUMBER OF OBSERVATIONS: 757983
            STATION_NUMBER     TEMP_F    TEMP_C
YEAR MONTH                                     
2019 4               29440  42.472030  5.817794
1990 4               29440  41.918084  5.510047
1989 4               29440  41.369647  5.205360
2011 4               29440  41.290730  5.161517
2004 4               29440  41.249676  5.138709




We will use the `glob()` function from the module `glob` to list our input files. 

In [49]:
import glob

In [50]:
file_list = glob.glob(r'data/0*txt')

<div class="alert alert-info">

**Note**

Note that we're using the \* character as a wildcard, so any file that starts with `data/0` and ends with `txt` will be added to the list of files we will iterate over. We specifically use `data/0` as the starting part of the file names to avoid having our metadata files included in the list!
</div>

In [51]:
print("Number of files in the list", len(file_list))
print(file_list)

Number of files in the list 15
['data\\028360.txt', 'data\\028690.txt', 'data\\028750.txt', 'data\\028970.txt', 'data\\029070.txt', 'data\\029110.txt', 'data\\029170.txt', 'data\\029350.txt', 'data\\029440.txt', 'data\\029500.txt', 'data\\029700.txt', 'data\\029720.txt', 'data\\029740.txt', 'data\\029810.txt', 'data\\029820.txt']


Now, you should have all the relevant file names in a list, and we can loop over the list using a for-loop:

In [52]:
for fp in file_list:
    print(fp)

data\028360.txt
data\028690.txt
data\028750.txt
data\028970.txt
data\029070.txt
data\029110.txt
data\029170.txt
data\029350.txt
data\029440.txt
data\029500.txt
data\029700.txt
data\029720.txt
data\029740.txt
data\029810.txt
data\029820.txt


In [53]:
# Repeat the analysis steps for each input file:
for fp in file_list:

    # Read selected columns of  data using varying amount of spaces as separator and specifying * characters as NoData values
    data = pd.read_csv(fp, delim_whitespace=True, usecols=['USAF','YR--MODAHRMN', 'DIR', 'SPD', 'GUS','TEMP', 'MAX', 'MIN'], na_values=['*', '**', '***', '****', '*****', '******'])

    # Rename the columns
    new_names = {'USAF':'STATION_NUMBER','YR--MODAHRMN': 'TIME', 'SPD': 'SPEED', 'GUS': 'GUST', 'TEMP':'TEMP_F'}
    data = data.rename(columns=new_names)

    #Print info about the current input file:
    print("STATION NUMBER:", data.at[0,"STATION_NUMBER"])
    print("NUMBER OF OBSERVATIONS:", len(data))

    # Create column
    col_name = 'TEMP_C'
    data[col_name] = None

    # Convert tempetarues from Fahrenheits to Celsius
    data["TEMP_C"] = data["TEMP_F"].apply(fahr_to_celsius)

    # Convert TIME to string 
    data['TIME_STR'] = data['TIME'].astype(str)

    # Parse year and month
    data['MONTH'] = data['TIME_STR'].str.slice(start=5, stop=6).astype(int)
    data['YEAR'] = data['TIME_STR'].str.slice(start=0, stop=4).astype(int)

    # Extract observations for the months of April 
    aprils = data[data['MONTH']==4]

    # Take a subset of columns
    aprils = aprils[['STATION_NUMBER','TEMP_F', 'TEMP_C', 'YEAR', 'MONTH']]

    # Group by year and month
    grouped = aprils.groupby(by=["YEAR", "MONTH"])

    # Get mean values for each group
    monthly_mean = grouped.mean()

    # Print info
    print(monthly_mean.sort_values(by="TEMP_C", ascending=False).head(5))
    print("\n")

STATION NUMBER: 28360
NUMBER OF OBSERVATIONS: 193825
            STATION_NUMBER     TEMP_F    TEMP_C
YEAR MONTH                                     
1937 4               28360  38.738095  3.743386
2011 4               28360  36.699571  2.610873
1921 4               28360  36.622222  2.567901
2002 4               28360  36.500000  2.500000
2019 4               28360  34.979138  1.655076


STATION NUMBER: 28690
NUMBER OF OBSERVATIONS: 542788
            STATION_NUMBER     TEMP_F    TEMP_C
YEAR MONTH                                     
2011 4               28690  35.430640  1.905911
2019 4               28690  35.215114  1.786174
2016 4               28690  35.031103  1.683946
1989 4               28690  34.612766  1.451537
2002 4               28690  34.279855  1.266586


STATION NUMBER: 28750
NUMBER OF OBSERVATIONS: 474562
            STATION_NUMBER     TEMP_F    TEMP_C
YEAR MONTH                                     
1989 4               28750  39.008403  3.893557
1983 4               

How about now, how did April 2019 rank across different stations?